In [1]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from datetime import datetime
import threading
lock = threading.RLock()

In [2]:
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])

proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
proxiesDf.head()


headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}

def get_proxie():
    return(
        {
            "http": proxiesDf.sample(1).loc[:,'IP Address'].values[0]
        }    
    )

In [3]:
rawVideos = pd.read_pickle('../datasets/videosData.pkl')
print(len(rawVideos))
rawVideos.head(3)

31583


,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate,text
-eLgCDFKD7s,The Art of living with Leopards,Nikit Surve,TEDxVileParle,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,1 week ago,10,0,1006,"[TEDxTalks, English, Global Issues, Animals]",544,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Nikit talks about wildlife and his work within...,1.57697e+09,I don't have never was in Mumbai so muchall of...
DF4ipJE_t_A,Paisajes invisibles,Francisco García & Manuel Bugallo,TEDxGalicia,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,1 week ago,21,0,723,"[TEDxTalks, Spanish, Global Issues, Environment]",761,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Fran y Manuel nos ofrecen una mirada diferente...,1.57699e+09,I always loved the Mediterranean maybe because...
587FCkioktM,Ce que j’ai appris de 15 000km à vélo,Pierre Bouillot,TEDxINSA,587FCkioktM,Ce que j’ai appris de 15 000km à vélo | Pierre...,1 week ago,6,0,890,"[TEDxTalks, French, Life, Adventure, Explorati...",283,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","Diplômé de l'INSA Lyon en 2016, Pierre Bouillo...",1.57699e+09,we're fine great started you go open your eye...


In [4]:
rawVideos.loc[:, 'publishDate'] = np.nan
rawVideos.head(2)

,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate,text,publishDate
-eLgCDFKD7s,The Art of living with Leopards,Nikit Surve,TEDxVileParle,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,1 week ago,10,0,1006,"[TEDxTalks, English, Global Issues, Animals]",544,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Nikit talks about wildlife and his work within...,1.57697e+09,I don't have never was in Mumbai so muchall of...,NaN
DF4ipJE_t_A,Paisajes invisibles,Francisco García & Manuel Bugallo,TEDxGalicia,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,1 week ago,21,0,723,"[TEDxTalks, Spanish, Global Issues, Environment]",761,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Fran y Manuel nos ofrecen una mirada diferente...,1.57699e+09,I always loved the Mediterranean maybe because...,NaN


In [17]:
headersYouTube = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
    'X-YouTube-Client-Name': '1',
    'X-YouTube-Client-Version': '2.20190509',
}


import urllib.parse as urlparse


def getVideoData(video_id):
    s = requests.Session()
    proxies = get_proxie()
    videoUrl = 'https://www.youtube.com/watch?v='+video_id+'&pbj=1'
    response = s.get(videoUrl, headers=headersYouTube, proxies=proxies)
    data = json.loads(response.text)

    publishDate =  data[3]['playerResponse']['microformat']['playerMicroformatRenderer']['publishDate']
    publishDate = pd.to_datetime(publishDate, format='%Y-%m-%d', errors='ignore')
#     display(dataFinal)
    with lock:
        rawVideos.loc[video_id, 'publishDate'] = publishDate


getVideoData('L0qtA5iMrAc')
# getVideoData('6Zex8K6h_jk')
# getVideoData('agsTBnU8plU')
# getVideoData('nVFc6iWk5b0') #no tiene ingles
# getVideoData('U6M5iX9qWh8')

In [6]:
def process_id(vidId):
    """process a single ID"""
    try:
        getVideoData(vidId)
    except:
        print('ERROR IN', vidId)
    return True

def process_range(id_range, store=None):
    total = len(id_range)
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    curr = 0
    for id in id_range:
        print(str(curr)+' of '+str(total))
        curr += 1
        store[id] = process_id(id)
    return store

from threading import Thread

def threaded_process_range(nthreads, id_range):
    """process the id range in a specified number of threads"""
    store = {}
    threads = []
    # create the threads
    for i in range(nthreads):
        ids = id_range[i::nthreads]
        t = Thread(target=process_range, args=(ids,store))
        threads.append(t)

    # start the threads
    [ t.start() for t in threads ]
    # wait for the threads to finish
    [ t.join() for t in threads ]
    return store

threaded_process_range(
    15,
    rawVideos.index.values,
)

0 of 2106
0 of 2106
0 of 2106
0 of 2106
0 of 2106
0 of 2106
0 of 2106
0 of 21060 of 2105

0 of 2105
0 of 2105
0 of 2105
0 of 2105
0 of 2105
0 of 2105
1 of 2106
1 of 2106
1 of 2106
1 of 2106
1 of 2106
1 of 2105
1 of 2105
1 of 2105
1 of 2105
1 of 2105
1 of 2106
1 of 2106
1 of 2106
1 of 2105
1 of 2105
2 of 2106
2 of 2106
2 of 2106
2 of 2106
2 of 2106
2 of 2105
2 of 2105
2 of 2105
2 of 2105
2 of 2106
2 of 2105
2 of 2105
2 of 2106
2 of 2106
2 of 2105
3 of 2106
3 of 2106
3 of 2106
3 of 2106
3 of 2105
3 of 2106
3 of 2105
3 of 2105
3 of 2106
3 of 2105
3 of 2105
3 of 2105
4 of 2106
3 of 2105
3 of 2106
4 of 2106
3 of 2106
4 of 2106
4 of 2106
4 of 2105
4 of 2106
4 of 2105
4 of 2105
4 of 2106
4 of 2105
4 of 2105
5 of 2106
4 of 2105
4 of 2106
4 of 2105
5 of 2106
4 of 2106
5 of 2106
5 of 2106
5 of 2105
5 of 2106
5 of 2105
5 of 2105
5 of 2105
6 of 2106
5 of 2105
6 of 2106
5 of 21055 of 2105

5 of 2106
5 of 2106
6 of 2105
5 of 2106
6 of 2106
6 of 2106
6 of 2106
6 of 21056 of 2105

6 of 2105
7 of 2106


52 of 2105
50 of 2106
51 of 2106
50 of 2105
51 of 2106
51 of 2106
51 of 2106
49 of 2105
50 of 2105
52 of 2105
51 of 2106
53 of 2105
50 of 2105
51 of 2106
48 of 2106
53 of 2105
52 of 2106
50 of 2105
52 of 2106
52 of 2106
51 of 2106
51 of 2105
51 of 2105
52 of 2106
53 of 2105
49 of 2106
52 of 2106
54 of 2105
52 of 2106
51 of 2105
54 of 2105
51 of 2105
53 of 2106
52 of 2105
52 of 2106
53 of 2106
53 of 2106
54 of 2105
53 of 2106
52 of 2105
50 of 2106
55 of 2105
53 of 2106
53 of 2106
52 of 2105
52 of 2105
53 of 210654 of 2106

55 of 2105
53 of 2105
53 of 2105
54 of 2106
55 of 2105
54 of 2106
54 of 2106
51 of 2106
56 of 2105
54 of 2106
54 of 2106
53 of 2105
54 of 2106
53 of 2105
54 of 2105
55 of 2106
55 of 2106
55 of 2106
55 of 2106
52 of 2106
57 of 2105
56 of 2105
56 of 2105
54 of 2105
55 of 2106
54 of 2105
55 of 2106
55 of 2106
55 of 2105
56 of 2106
56 of 2106
56 of 2106
56 of 2106
56 of 2106
58 of 2105
54 of 2105
57 of 2105
53 of 2106
56 of 2106
55 of 2105
57 of 2105
56 of 2105
56 of 2106

99 of 2105
99 of 2105
99 of 2105
100 of 2106
101 of 2106
101 of 2106
101 of 2105
102 of 2106
101 of 2105102 of 2105

101 of 2106
101 of 2106
103 of 2105
101 of 2106
100 of 2105
100 of 2105
100 of 2105
97 of 2106
101 of 2106
102 of 2106
102 of 2106
102 of 2105
103 of 2105
104 of 2105
102 of 2106
101 of 2105
101 of 2105
102 of 2105
102 of 2106
101 of 210598 of 2106

102 of 2106
103 of 2106
103 of 2106
102 of 2106
103 of 2105
103 of 2106
105 of 2105
104 of 2105
102 of 2105
102 of 2105
103 of 2106
103 of 2105
102 of 2105
104 of 2106
99 of 2106103 of 2106

103 of 2106
104 of 2106
103 of 2106
104 of 2105
106 of 2105
103 of 2105
105 of 2105
104 of 2106
104 of 2105
103 of 2105
103 of 2105
104 of 2106
100 of 2106
104 of 2106
104 of 2106
105 of 2106
105 of 2106
105 of 2105
104 of 2106
107 of 2105
106 of 2105
105 of 2106
105 of 2105
101 of 2106
104 of 2105
105 of 2106
105 of 2106
106 of 2106
104 of 2105
105 of 2106
104 of 2105
106 of 2106
106 of 2105
108 of 2105
105 of 2106
107 of 2105
106 of 210

148 of 2105
145 of 2106
144 of 2105
145 of 2106
144 of 2106
145 of 2106
147 of 2105
148 of 2106
147 of 2106
147 of 2106
144 of 2105
149 of 2105
149 of 2105
147 of 2105
145 of 2105
145 of 2105
146 of 2106
145 of 2106
146 of 2106
146 of 2106
146 of 2106
148 of 2105
149 of 2106
148 of 2106
148 of 2106
150 of 2105
148 of 2105
150 of 2105
145 of 2105
146 of 2105
147 of 2106
146 of 2105
147 of 2106
147 of 2106
147 of 2106146 of 2106

149 of 2105
149 of 2106150 of 2106

149 of 2106
151 of 2105
149 of 2105
151 of 2105
146 of 2105
147 of 2105
148 of 2106
148 of 2106
148 of 2106
148 of 2106
147 of 2105
147 of 2106
150 of 2106
151 of 2106150 of 2106

152 of 2105
150 of 2105
149 of 2106
150 of 2105
148 of 2105
147 of 2105
149 of 2106
152 of 2105
149 of 2106
149 of 2106
151 of 2106
148 of 2106
148 of 2105
151 of 2106
152 of 2106
150 of 2106
153 of 2105
151 of 2105
150 of 2106
153 of 2105
148 of 2105
149 of 2105
150 of 2106
151 of 2105
152 of 2106
150 of 2106
149 of 2105
152 of 2106
149 of 2106
153 

192 of 2105
194 of 2105
191 of 2105
189 of 2105
190 of 2106
194 of 2105
193 of 2106
191 of 2106
192 of 2105
192 of 2106
191 of 2106
192 of 2106
192 of 2105
190 of 2105
193 of 2106
191 of 2105
193 of 2105
195 of 2105
190 of 2106
195 of 2105
191 of 2106
192 of 2106
194 of 2106
192 of 2106
193 of 2105
193 of 2106
193 of 2105
194 of 2106
191 of 2106
196 of 2105
192 of 2105
194 of 2105
191 of 2105
193 of 2106
192 of 2106
193 of 2106
195 of 2106
196 of 2105
193 of 2106
194 of 2105
194 of 2106
194 of 2105
195 of 2106
193 of 2105
193 of 2106
194 of 2106
192 of 2106
196 of 2106
195 of 2105
192 of 2105
197 of 2105
197 of 2105
194 of 2106
194 of 2106
195 of 2106
195 of 2105
195 of 2105
194 of 2106
193 of 2106
194 of 2105195 of 2106

197 of 2106
196 of 2106
193 of 2105
196 of 2105
195 of 2106
198 of 2105
198 of 2105
196 of 2105
196 of 2105
195 of 2106
196 of 2106
195 of 2106
194 of 2106
195 of 2105
194 of 2105
196 of 2106
197 of 2106
198 of 2106
196 of 2106
199 of 2105
197 of 2105
196 of 2106197 o

...

1944 of 2106
1928 of 2106
1936 of 2106
1940 of 2105
1939 of 2105
1933 of 2105
1936 of 2106
1931 of 2105
1943 of 2106
1936 of 2106
1946 of 2106
1945 of 2106
1947 of 2105
1945 of 2106
1942 of 2105
1929 of 2106
1942 of 2105
1941 of 2105
1937 of 2106
1940 of 2105
1934 of 2105
1944 of 2106
1946 of 2106
1937 of 2106
1947 of 2106
1932 of 2105
1943 of 2105
1948 of 21051937 of 2106

1930 of 2106
1943 of 2105
1938 of 2106
1946 of 2106
1942 of 2105
1941 of 2105
1935 of 2105
1945 of 2106
1947 of 2106
1938 of 2106
1948 of 2106
1949 of 2105
1933 of 2105
1931 of 2106
1944 of 2105
1939 of 2106
1938 of 2106
1942 of 2105
1943 of 2105
1946 of 2106
1947 of 2106
1944 of 2105
1936 of 2105
1939 of 2106
1949 of 2106
1934 of 2105
1948 of 2106
1950 of 2105
1940 of 2106
1932 of 2106
1943 of 2105
1945 of 2105
1947 of 2106
1939 of 21061944 of 2105

1937 of 2105
1948 of 2106
1945 of 2105
1940 of 2106
1949 of 2106
1950 of 2106
1935 of 2105
1951 of 2105
1941 of 2106
1944 of 2105
1948 of 2106
1946 of 2105
1933 of 2106

1973 of 2105
1990 of 2106
1984 of 2105
1982 of 2105
1969 of 2106
1978 of 2106
1983 of 2105
1982 of 2105
1985 of 2106
1980 of 2106
1974 of 2105
1990 of 2105
1987 of 2106
1977 of 21061988 of 2106

1974 of 2105
1979 of 2106
1985 of 2105
1970 of 2106
1983 of 2105
1986 of 2106
1991 of 2106
1984 of 2105
1981 of 2106
1983 of 2105
1975 of 2105
1989 of 2106
1988 of 2106
1991 of 2105
1980 of 2106
1975 of 2105
1978 of 2106
1986 of 2105
1992 of 2106
1985 of 2105
1971 of 2106
1987 of 2106
1984 of 2105
1982 of 2106
1976 of 2105
1989 of 2106
1984 of 2105
1990 of 2106
1992 of 2105
1979 of 2106
1987 of 2105
1976 of 2105
1993 of 2106
1981 of 2106
1986 of 2105
1972 of 2106
1988 of 2106
1983 of 2106
1985 of 2105
1985 of 2105
1990 of 2106
1993 of 2105
1980 of 2106
1988 of 2105
1991 of 2106
1977 of 2105
1977 of 2105
1994 of 2106
1982 of 2106
1987 of 2105
1973 of 2106
1986 of 2105
1991 of 2106
1984 of 2106
1989 of 2106
1986 of 2105
1994 of 2105
1981 of 2106
1992 of 2106
1978 of 2105
1988 of 2105
1989 of 2105

2026 of 2105
2018 of 2106
2016 of 2105
2022 of 2105
2023 of 2105
2022 of 2106
2026 of 2105
2012 of 2106
2031 of 2106
2032 of 2105
2032 of 2106
2020 of 2106
2029 of 2106
2017 of 2105
2027 of 2105
2019 of 2106
2024 of 2105
2023 of 2105
2023 of 2106
2027 of 2105
2032 of 2106
2017 of 2105
2033 of 2105
2021 of 2106
2013 of 2106
2027 of 2106
2033 of 2106
2030 of 2106
2018 of 2105
2028 of 2105
2020 of 2106
2024 of 2105
2025 of 2105
2034 of 2105
2024 of 2106
2022 of 2106
2028 of 2105
2033 of 2106
2028 of 2106
2018 of 2105
2031 of 2106
2014 of 2106
2019 of 2105
2029 of 2105
2025 of 2105
2034 of 2106
2021 of 2106
2026 of 2105
2023 of 2106
2034 of 2106
2035 of 2105
2025 of 2106
2019 of 2105
2015 of 2106
2032 of 2106
2029 of 2106
2029 of 2105
2030 of 2105
2026 of 2105
2035 of 2106
2022 of 2106
2020 of 2105
2027 of 2105
2036 of 2105
2030 of 2105
2020 of 2105
2016 of 2106
2035 of 2106
2031 of 2105
2024 of 2106
2026 of 2106
2030 of 2106
2033 of 2106
2027 of 2105
2036 of 2106
2023 of 2106
2028 of 2105

2069 of 21052065 of 2105

2060 of 2106
2070 of 2105
2069 of 2106
2064 of 2106
2074 of 2106
2073 of 2105
2052 of 2106
2060 of 2105
2058 of 2105
2063 of 2106
2066 of 2105
2066 of 2105
2075 of 2106
2069 of 2106
2070 of 2105
2061 of 2106
2071 of 2105
2070 of 2106
2075 of 2106
2074 of 2105
2061 of 2105
2053 of 2106
2065 of 2106
2076 of 2106
2064 of 2106
2070 of 2106
2067 of 2105
2071 of 2105
2059 of 2105
2062 of 2106
2072 of 2105
2067 of 2105
2076 of 2106
2054 of 2106
2071 of 2106
2066 of 2106
2062 of 2105
2075 of 2105
2077 of 2106
2065 of 2106
2068 of 2105
2072 of 2105
2068 of 2105
2063 of 2106
2060 of 2105
2071 of 2106
2073 of 2105
2077 of 2106
2072 of 2106
2063 of 2105
2067 of 2106
2078 of 2106
2076 of 2105
2066 of 2106
2055 of 2106
2073 of 2105
2064 of 2106
2069 of 2105
2061 of 2105
2078 of 2106
2072 of 2106
2069 of 2105
2064 of 2105
2073 of 2106
2074 of 2105
2067 of 2106
2077 of 2105
2079 of 2106
2068 of 2106
2056 of 21062070 of 2105

2065 of 2106
2074 of 2105
2073 of 2106
2065 of 2105

{'587FCkioktM': True,
 'DF4ipJE_t_A': True,
 'm5jeaTARaUE': True,
 'xUZf2ZUl1xA': True,
 '-eLgCDFKD7s': True,
 'mKbbC9zx8oA': True,
 'pk5T1sLhO5U': True,
 'f-_GsvYuY6g': True,
 'Dg5ifmBfBbg': True,
 '10S4vIg6q-0': True,
 'P50Rk_WozbU': True,
 'UY5bDA63b7k': True,
 '12D6gTp2924': True,
 'C-MoChugxZk': True,
 'xy9hkDH2ZV8': True,
 'Uz7vjIOX4s8': True,
 'xwqivojxXME': True,
 'uNIATA9U-Xg': True,
 'ApbqizI_0rs': True,
 'OtezRQTSDog': True,
 'SpUhPGSYF-g': True,
 'vgungeg3BvE': True,
 'NyEdORh1i6o': True,
 'eaxVmZoxj6Y': True,
 'kiuWfrvywBQ': True,
 '6bwQPTA6Q0g': True,
 'lnY7zn18rrg': True,
 'iPAgepy1aOw': True,
 'b0EdU-mTkZA': True,
 'O8R7M2xLKAU': True,
 'DzGnSy-0gSI': True,
 '7vDjk4Ue3q8': True,
 'qs0IjZk35xs': True,
 'q5slk0ScUXU': True,
 'QIULhTlv9xU': True,
 'VUnAUWTKzp0': True,
 '7ZHIsyenoZo': True,
 'VCZ5jidqrZw': True,
 'vDko82c5DhQ': True,
 'nZGvRoJ7isc': True,
 'UCNl8cJh_eY': True,
 'vEJks0vSsBQ': True,
 'aIRsv1bWib4': True,
 'YwUDhttT_vE': True,
 'LeYYOcc0Lqc': True,
 '4RwYYh1g

In [38]:
# rawVideos[rawVideos.index.duplicated()]
# df1.index = df1.index.values
df1

,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate,text,publishDate
-eLgCDFKD7s,The Art of living with Leopards,Nikit Surve,TEDxVileParle,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,1 week ago,10,0,1006,"[TEDxTalks, English, Global Issues, Animals]",544,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Nikit talks about wildlife and his work within...,1.57697e+09,I don't have never was in Mumbai so muchall of...,2019-05-17 00:00:00
DF4ipJE_t_A,Paisajes invisibles,Francisco García & Manuel Bugallo,TEDxGalicia,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,1 week ago,21,0,723,"[TEDxTalks, Spanish, Global Issues, Environment]",761,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Fran y Manuel nos ofrecen una mirada diferente...,1.57699e+09,I always loved the Mediterranean maybe because...,2019-05-17 00:00:00
587FCkioktM,Ce que j’ai appris de 15 000km à vélo,Pierre Bouillot,TEDxINSA,587FCkioktM,Ce que j’ai appris de 15 000km à vélo | Pierre...,1 week ago,6,0,890,"[TEDxTalks, French, Life, Adventure, Explorati...",283,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","Diplômé de l'INSA Lyon en 2016, Pierre Bouillo...",1.57699e+09,we're fine great started you go open your eye...,2019-05-17 00:00:00
UY5bDA63b7k,O Poder do Imprevisto na Música,André Santos,TEDxFunchal,UY5bDA63b7k,O Poder do Imprevisto na Música | André Santos...,1 week ago,44,1,1032,"[TEDxTalks, Portuguese, Art, Connection, Initi...",1164,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","André Santos é um guitarrista madeirense, que ...",1.57699e+09,[Applause] we are ready for the court on the...,2019-05-17 00:00:00
xUZf2ZUl1xA,My mom read the news every day. How journalism...,Florence Rogers,TEDxValleyHS,xUZf2ZUl1xA,My mom read the news every day. How journalism...,1 week ago,3,0,880,"[TEDxTalks, English, Humanities, Humanity, Ide...",108,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",My mom read the news every day. How journalism...,1.57699e+09,[Applause]so I just came back from the Isle of...,2019-05-17 00:00:00
m5jeaTARaUE,The Only Way To Predict The Future Is To Creat...,Bethany Girma,TEDxValleyHS,m5jeaTARaUE,The Only Way To Predict The Future Is To Creat...,1 week ago,23,8,514,"[TEDxTalks, English, Life, Achievement, Advent...",413,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",The Only Way To Predict The Future Is To Creat...,1.57699e+09,[Applause]good morning ladies and gentlemen my...,2019-05-17 00:00:00
12D6gTp2924,Come e perchè mantenere pulito lo Spazio,Rosario Nasca,TEDxBarletta,12D6gTp2924,Come e perchè mantenere pulito lo Spazio | Ros...,1 week ago,20,0,851,"[TEDxTalks, Italian, Global Issues, Environmen...",373,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Un viaggio nello spazio alla ricerca dell'impr...,1.57699e+09,[Music] [Applause] imagine yourself sitting ...,2019-05-17 00:00:00
P50Rk_WozbU,Banking for the Unbanked - Katharine Budd,Katharine Budd,TEDxAlWaslWomen,P50Rk_WozbU,Banking for the Unbanked - Katharine Budd | Ka...,1 week ago,3,0,923,"[TEDxTalks, English, Global Issues, Developing...",171,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","Katharine is co-founder of Now Money, a platfo...",1.57697e+09,good morning everybody I'm here today totalk t...,2019-05-17 00:00:00
mKbbC9zx8oA,Turning Loss into Legacy - Danielle Wilson Naqvi,Danielle Wilson Naqvi,TEDxAlWaslWomen,mKbbC9zx8oA,Turning Loss into Legacy - Danielle Wilson Naq...,1 week ago,0,0,1121,"[TEDxTalks, English, Life, Children, Death, De...",137,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","In this talk, Danielle Wilson Naqvi will be sh...",1.57699e+09,my story of what-ifs is going to take uson a j...,2019-05-17 00:00:00
Dg5ifmBfBbg,Gender equality: taking action at a local level,Derek O'Connell,TEDxMackay,Dg5ifmBfBbg,Gender equality: taking action at a local leve...,1 week ago,3,1,995,"[TEDxTalks

In [31]:
rawVideos

,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate,text,publishDate
-eLgCDFKD7s,The Art of living with Leopards,Nikit Surve,TEDxVileParle,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,1 week ago,10,0,1006,"[TEDxTalks, English, Global Issues, Animals]",544,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Nikit talks about wildlife and his work within...,1.57697e+09,I don't have never was in Mumbai so muchall of...,2019-05-17 00:00:00
DF4ipJE_t_A,Paisajes invisibles,Francisco García & Manuel Bugallo,TEDxGalicia,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,1 week ago,21,0,723,"[TEDxTalks, Spanish, Global Issues, Environment]",761,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Fran y Manuel nos ofrecen una mirada diferente...,1.57699e+09,I always loved the Mediterranean maybe because...,2019-05-17 00:00:00
587FCkioktM,Ce que j’ai appris de 15 000km à vélo,Pierre Bouillot,TEDxINSA,587FCkioktM,Ce que j’ai appris de 15 000km à vélo | Pierre...,1 week ago,6,0,890,"[TEDxTalks, French, Life, Adventure, Explorati...",283,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","Diplômé de l'INSA Lyon en 2016, Pierre Bouillo...",1.57699e+09,we're fine great started you go open your eye...,2019-05-17 00:00:00
UY5bDA63b7k,O Poder do Imprevisto na Música,André Santos,TEDxFunchal,UY5bDA63b7k,O Poder do Imprevisto na Música | André Santos...,1 week ago,44,1,1032,"[TEDxTalks, Portuguese, Art, Connection, Initi...",1164,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","André Santos é um guitarrista madeirense, que ...",1.57699e+09,[Applause] we are ready for the court on the...,2019-05-17 00:00:00
xUZf2ZUl1xA,My mom read the news every day. How journalism...,Florence Rogers,TEDxValleyHS,xUZf2ZUl1xA,My mom read the news every day. How journalism...,1 week ago,3,0,880,"[TEDxTalks, English, Humanities, Humanity, Ide...",108,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",My mom read the news every day. How journalism...,1.57699e+09,[Applause]so I just came back from the Isle of...,2019-05-17 00:00:00
m5jeaTARaUE,The Only Way To Predict The Future Is To Creat...,Bethany Girma,TEDxValleyHS,m5jeaTARaUE,The Only Way To Predict The Future Is To Creat...,1 week ago,23,8,514,"[TEDxTalks, English, Life, Achievement, Advent...",413,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",The Only Way To Predict The Future Is To Creat...,1.57699e+09,[Applause]good morning ladies and gentlemen my...,2019-05-17 00:00:00
12D6gTp2924,Come e perchè mantenere pulito lo Spazio,Rosario Nasca,TEDxBarletta,12D6gTp2924,Come e perchè mantenere pulito lo Spazio | Ros...,1 week ago,20,0,851,"[TEDxTalks, Italian, Global Issues, Environmen...",373,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Un viaggio nello spazio alla ricerca dell'impr...,1.57699e+09,[Music] [Applause] imagine yourself sitting ...,2019-05-17 00:00:00
P50Rk_WozbU,Banking for the Unbanked - Katharine Budd,Katharine Budd,TEDxAlWaslWomen,P50Rk_WozbU,Banking for the Unbanked - Katharine Budd | Ka...,1 week ago,3,0,923,"[TEDxTalks, English, Global Issues, Developing...",171,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","Katharine is co-founder of Now Money, a platfo...",1.57697e+09,good morning everybody I'm here today totalk t...,2019-05-17 00:00:00
mKbbC9zx8oA,Turning Loss into Legacy - Danielle Wilson Naqvi,Danielle Wilson Naqvi,TEDxAlWaslWomen,mKbbC9zx8oA,Turning Loss into Legacy - Danielle Wilson Naq...,1 week ago,0,0,1121,"[TEDxTalks, English, Life, Children, Death, De...",137,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","In this talk, Danielle Wilson Naqvi will be sh...",1.57699e+09,my story of what-ifs is going to take uson a j...,2019-05-17 00:00:00
Dg5ifmBfBbg,Gender equality: taking action at a local level,Derek O'Connell,TEDxMackay,Dg5ifmBfBbg,Gender equality: taking action at a local leve...,1 week ago,3,1,995,"[TEDxTalks

In [39]:
df1.to_pickle('../datasets/videosData.pkl')